# 2.3 What are autoencoders to GANs?
- Key distinction
 - autoencoders: end-to-end train the whole network with one loss function.
 - GANs: distinct loss functions for the Generator and the Discriminator.
   - GANs have two competing objectives that cannot be written in one function.
- GANs in Action本の全部実践notebookはここにある。https://github.com/GANs-in-Action/gans-in-action
- また、たくさん種類のGANの実現はここにある。
 - Kerasで：https://github.com/eriklindernoren/Keras-GAN
 - PyTorchで：https://github.com/eriklindernoren/PyTorch-GAN

# 2.7 Variational Autoencoder code

## Step 1: Standard imports and Setting hyperparameters

In [1]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf

from keras.layers import Input, Dense, Lambda, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
batch_size = 100
orignal_dim = 784 # 28*28
latent_dim = 2 # bivariate Gaussian.つまりz_meanやz_log_varのサイズは全部1*2です。ちょっと小さすぎじゃない？
intermediate_dim = 256 # input層やlatent層の中間層のサイズ。
epochs = 50
epsilon_std = 1.0

## Step 2: Creating the sampling helper function
- encoderの結果としてのz_mean,z_log_varからsamplingする。
- reparametrization trickを使っている。VAE & GAN勉強会資料を参考してください。https://github.com/youngsend/DeepLearningAndKeras/blob/master/Deep-Learning-Ian-Goodfellow/Chapter20.9-20.10_VAE_GAN.pdf
- 名前の通り、$z\_log\_var=log(\sigma^2)=2log(\sigma)$.
 - だから$y=\mu+\sigma z=\mu + exp(z\_log\_var / 2)z$.
 - $z\sim\mathcal{N}(z;0,1)$.

In [4]:
def sampling(args: tuple):
    z_mean, z_log_var = args
    # K.shape(z_mean)[0]はbatch_sizeです。つまり例えばinputの画像数。epsilonのサイズはbatch_size*latent_dim=100*2
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                             stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon